In [3]:
import pandas as pd
import mysql.connector

In [5]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Azerty2024/",
  database="m1_ds2e"
)

c = mydb.cursor()

c.execute("SELECT * FROM sample_superstore")

docs = c.fetchall()

columns = [description[0] for description in c.description]
df = pd.DataFrame(docs, columns=columns)

In [8]:
# 1: How many orders were placed in total?
print(f"Il y a eu {df.shape[0]} commandes au total")

Il y a eu 175 commandes au total


In [47]:
print(df.columns)

Index(['Row ID', 'Order Priority', 'Discount', 'Unit Price', 'Shipping Cost',
       'Customer ID', 'Customer Name', 'Ship Mode', 'Customer Segment',
       'Product Category', 'Product Sub-Category', 'Product Container',
       'Product Name', 'Product Base Margin', 'Region', 'State or Province',
       'City', 'Postal Code', 'Order Date', 'Ship Date', 'Profit',
       'Quantity ordered new', 'Sales', 'Order ID'],
      dtype='object')


In [10]:
# 2: What is the total sales amount?
c.execute("SELECT SUM(Sales) AS TotalSales FROM sample_superstore")
total_sales = c.fetchone()[0]
print("Total des ventes:", total_sales)

Total des ventes: 130555.31000000006


In [12]:
#3: What is the average profit per order?
c.execute("SELECT AVG(Profit) AS MoySales FROM sample_superstore")
moy_sales = c.fetchone()[0]
print("Profit Moyen:", moy_sales)

Profit Moyen: 136.04226416285724


In [18]:
#4: Which customer has placed the most orders?
c.execute("""
    SELECT `Customer ID`, COUNT(*) AS OrderCount
    FROM sample_superstore
    GROUP BY `Customer ID`
    ORDER BY OrderCount DESC
    LIMIT 1
""")
customer_with_most_orders = c.fetchone()
if customer_with_most_orders:
    customer_id, order_count = customer_with_most_orders
    print(f"Le client avec l'ID {customer_id} a passé le plus de commandes, avec un total de {order_count} commandes.")


Le client avec l'ID 21 a passé le plus de commandes, avec un total de 12 commandes.


In [20]:
#5: What is the total profit for each product category?

c.execute("""
    SELECT `Product Category`, SUM(Profit) AS TotalProfit
    FROM sample_superstore
    GROUP BY `Product Category`
""")
total_profit_by_category = c.fetchall()

for category, total_profit in total_profit_by_category:
    print(f"Catégorie de produit: {category}, Profit total: {total_profit}")


Catégorie de produit: Office Supplies, Profit total: 11605.983540000001
Catégorie de produit: Technology, Profit total: 10225.340091999999
Catégorie de produit: Furniture, Profit total: 1976.0725965000036


In [28]:
# 6: What is the average discount percentage?
c.execute("""
    SELECT AVG(Discount / Sales) * 100 AS AvgDiscountPct
    From sample_superstore
""")

average_discount_percentage = c.fetchone()[0]
print("Pourcentage moyen de remise:", average_discount_percentage)

Pourcentage moyen de remise: 0.0975737182075381


In [29]:
#7: Which region has the highest total sales?

c.execute("""
    SELECT Region, SUM(Sales) AS TotSales
    From sample_superstore
    GROUP BY Region
    ORDER BY TotSales DESC
""")

sales_by_region = c.fetchall()

for region, total_sales in sales_by_region:
    print(f"Région: {region}, Ventes totales: {total_sales}")

Région: West, Ventes totales: 75678.49999999999
Région: East, Ventes totales: 38033.84999999999
Région: Central, Ventes totales: 14261.619999999999
Région: South, Ventes totales: 2581.34


In [33]:
#8: How many orders were shipped using each shipping mode?
c.execute("""
    SELECT `Ship Mode`, COUNT(*) AS OrdersCount
    FROM sample_superstore
    GROUP BY `Ship Mode`
""")
orders_by_shipping_mode = c.fetchall()

for shipping_mode, orders_count in orders_by_shipping_mode:
    print(f"Mode de livraison : {shipping_mode}, Nombre de commandes : {orders_count}")

Mode de livraison : Regular Air, Nombre de commandes : 131
Mode de livraison : Express Air, Nombre de commandes : 23
Mode de livraison : Delivery Truck, Nombre de commandes : 21


In [36]:
#9: What is the total quantity ordered for each product sub-category?
c.execute("""
    SELECT `Product Sub-Category`, SUM(`Quantity ordered new`) AS TotalQuantity
    FROM sample_superstore
    GROUP BY `Product Sub-Category`
""")
quantity_by_subcategory = c.fetchall()

for x, total_quantity in quantity_by_subcategory:
    print(f"Sous-catégorie de produit : {x}, Quantité totale commandée : {total_quantity}")


Sous-catégorie de produit : Labels, Quantité totale commandée : 171
Sous-catégorie de produit : Pens & Art Supplies, Quantité totale commandée : 196
Sous-catégorie de produit : Paper, Quantité totale commandée : 286
Sous-catégorie de produit : Scissors, Rulers and Trimmers, Quantité totale commandée : 24
Sous-catégorie de produit : Telephones and Communication, Quantité totale commandée : 252
Sous-catégorie de produit : Office Machines, Quantité totale commandée : 123
Sous-catégorie de produit : Chairs & Chairmats, Quantité totale commandée : 166
Sous-catégorie de produit : Appliances, Quantité totale commandée : 184
Sous-catégorie de produit : Bookcases, Quantité totale commandée : 39
Sous-catégorie de produit : Tables, Quantité totale commandée : 50
Sous-catégorie de produit : Envelopes, Quantité totale commandée : 46
Sous-catégorie de produit : Office Furnishings, Quantité totale commandée : 233
Sous-catégorie de produit : Rubber Bands, Quantité totale commandée : 119
Sous-catégorie

In [51]:
#10: Which product has the highest profit margin?
c.execute("""
    SELECT `Product Name`, `Product Base Margin`
    FROM sample_superstore
    ORDER BY `Product Base Margin` DESC
    LIMIT 1
""")
product_with_highest_margin = c.fetchone()

if product_with_highest_margin:
    product_name = product_with_highest_margin[0]
    margin = product_with_highest_margin[1]
    print(f"Le produit avec la marge bénéficiaire la plus élevée est '{product_name}' avec une marge de {margin:.2f}.")
else:
    print("Aucun produit trouvé.")


Le produit avec la marge bénéficiaire la plus élevée est 'Accessory29' avec une marge de 0.83.


In [56]:
#11: What is the total sales amount for each year?

c.execute("""
    SELECT YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y')) AS Year, SUM(Sales) AS Total_Sales
    FROM sample_superstore
    GROUP BY YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y'))
    ORDER BY YEAR(STR_TO_DATE(`Order Date`, '%m/%d/%Y'))
""")

sales_by_year = c.fetchall()

for year, total_sales in sales_by_year:
    print("Year:", year, "- Total Sales:", total_sales)


Year: 2010 - Total Sales: 25377.779999999995
Year: 2011 - Total Sales: 12048.119999999997
Year: 2012 - Total Sales: 44893.209999999985
Year: 2013 - Total Sales: 48236.200000000004


In [48]:
c.execute("""
    SELECT `Customer Segment`, AVG(Sales) AS AvgSales
    FROM sample_superstore
    GROUP BY `Customer Segment`
    ORDER BY AvgSales DESC
    LIMIT 1
""")
segment_with_highest_avg_sales = c.fetchone()

if segment_with_highest_avg_sales:
    segment = segment_with_highest_avg_sales[0]
    avg_sales = segment_with_highest_avg_sales[1]
    print(f"Le segment de client avec le chiffre d'affaires moyen le plus élevé est '{segment}' avec une moyenne de ventes de {avg_sales:.2f}.")
else:
    print("Aucun segment trouvé.")


Le segment de client avec le chiffre d'affaires moyen le plus élevé est 'Small Business' avec une moyenne de ventes de 1041.56.
